In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.options.display.max_colwidth = None
pd.set_option("display.float_format", lambda x: '%.2f' % x)

from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
import sys

sys.path.insert(0, os.path.dirname(os.getcwd()))

from parameters.parameters import data_path, autogluon_params
from data_ingestion.ingest import get_data

# from autogluon.tabular import TabularDataset, TabularPredictor
from model_building.build_model import autogluon_model_build

In [4]:
churn_data = get_data(data_path)
churn_data.head()

,customer_id,credit_score,country,gender,age,tenure,balance,products_number,credit_card,active_member,estimated_salary,churn
0,15634602,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,15647311,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,15619304,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,15701354,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,15737888,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
churn_data = churn_data.drop('customer_id', axis='columns')

In [6]:
autogluon_params

{'save_path': '/Users/damilolaadesina/Workspace/Churn_Prediction/artefacts/model_autogluon',
 'time_limit': 60,
 'label': 'churn',
 'problem_type': 'binary'}

In [7]:
churn_data.columns

Index(['credit_score', 'country', 'gender', 'age', 'tenure', 'balance',
       'products_number', 'credit_card', 'active_member', 'estimated_salary',
       'churn'],
      dtype='object')

In [6]:
train_data, test_data, predictor = autogluon_model_build(churn_data, autogluon_params)

Beginning AutoGluon training ... Time limit = 60s
AutoGluon will save models to "/Users/damilolaadesina/Workspace/Churn_Prediction/artefacts/model_autogluon/"
AutoGluon Version:  0.7.0
Python Version:     3.9.16
Operating System:   Darwin
Platform Machine:   x86_64
Platform Version:   Darwin Kernel Version 22.4.0: Mon Mar  6 21:00:17 PST 2023; root:xnu-8796.101.5~3/RELEASE_X86_64
Train Data Rows:    6700
Train Data Columns: 10
Label Column: churn
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [1, 0]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                  

In [9]:
predictor.leaderboard(silent=True)

,model,score_val,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,NeuralNetTorch,0.858209,0.009633,6.849504,0.009633,6.849504,1,True,3
1,WeightedEnsemble_L2,0.858209,0.011144,7.012715,0.001511,0.163211,2,True,4
2,XGBoost,0.856716,0.006538,1.632799,0.006538,1.632799,1,True,1
3,LinearModel,0.808955,0.012122,2.220443,0.012122,2.220443,1,True,2


In [10]:
 predictor.leaderboard(test_data, silent=True)

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,XGBoost,0.867273,0.856716,0.021039,0.006538,1.632799,0.021039,0.006538,1.632799,1,True,1
1,NeuralNetTorch,0.866667,0.858209,0.022410,0.009633,6.849504,0.022410,0.009633,6.849504,1,True,3
2,WeightedEnsemble_L2,0.866667,0.858209,0.028776,0.011144,7.012715,0.006366,0.001511,0.163211,2,True,4
3,LinearModel,0.816970,0.808955,0.016421,0.012122,2.220443,0.016421,0.012122,2.220443,1,True,2


In [12]:
predictor.evaluate(test_data, silent=True)

{'accuracy': 0.8666666666666667,
 'balanced_accuracy': 0.7226903200618572,
 'mcc': 0.5277065492713041,
 'roc_auc': 0.860499364629129,
 'f1': 0.5872420262664164,
 'precision': 0.7399527186761229,
 'recall': 0.4867807153965785}

In [14]:
churn_data.head(3)

,credit_score,country,gender,age,tenure,balance,products_number,credit_card,active_member,estimated_salary,churn
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
